In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Import Data

#### 匯入測試資料，只取前10,000筆 chid

In [3]:
df_test = pd.read_csv(r'D:\Github\07_E_SUN_2021_Winter_AI\data\需預測的顧客名單及提交檔案範例.csv')
df_test = df_test[:10000].reset_index(drop=True)
df_test


,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26
...,...,...,...,...
9995,10203858,25,22,37
9996,10008113,22,48,15
9997,10053154,37,39,12
9998,10252782,26,18,12


#### 取出需要預測的類別

In [4]:
shop_tag_list = ['2', '6', '10', '12',
                '13', '15', '18', '19',
                '21', '22', '25', '26',
                '36', '37', '39', '48']

#### 可加可不加的參數，可以先限縮要取的資料欄位

In [5]:
col_list = ['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt','masts',
            'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam',
            'gender_code', 'age', 'primary_card']

In [6]:
reader = pd.read_csv(r'D:\Github\07_E_SUN_2021_Winter_AI\data\tbrain_cc_training_48tags_hash_final.csv',
                    error_bad_lines=False, # 會自動忽略錯誤row
                    # header=None, # 看資料有無欄位名稱
                    iterator=True,
                    # sep=',',
                    # usecols=col_list  #限縮要取的資料欄位
                    )
loop = True
n = 0
chunksize = 10 ** 5
chunks = []
while loop:
    try:
        chunk = reader.get_chunk(chunksize)
        chunk = chunk[chunk.chid.isin(df_test.chid)]    #只取測試資料的前10,000 chid
        chunk = chunk[chunk.shop_tag.isin(shop_tag_list)]   #指取要預測的消費類別
        chunks.append(chunk)
        n += 1  #可以用n來看大概的資料筆數
    except StopIteration:
        loop = False
        print("Iteration is stopped.")
df = pd.concat(chunks,ignore_index=True)


Iteration is stopped.


In [10]:
n

330

#### 可以推測，總資料大約3,300萬筆

# 2. Description

In [7]:
df

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1,10384918,10,1,1085.361456,0,0,0,1,0.00,...,2.0,3.0,15.0,1.0,99.0,30.0,38072.451849,0.0,3.0,1
1,1,10384918,48,4,8523.693513,0,0,0,4,0.00,...,2.0,3.0,15.0,1.0,99.0,30.0,38072.451849,0.0,3.0,1
2,1,10378106,2,2,2987.631549,2,0,0,0,1.00,...,2.0,5.0,5.0,1.0,99.0,30.0,48209.277501,1.0,3.0,1
3,1,10378106,12,2,8690.365708,1,1,0,0,0.25,...,2.0,5.0,5.0,1.0,99.0,30.0,48209.277501,1.0,3.0,1
4,1,10011300,2,3,4388.591772,3,0,0,0,1.00,...,1.0,2.0,13.0,1.0,3.0,30.0,112010.611717,1.0,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438940,24,10041911,2,7,14218.830280,7,0,0,0,1.00,...,1.0,4.0,5.0,1.0,2.0,30.0,96727.878252,0.0,4.0,1
438941,24,10041911,10,2,14646.184347,0,2,0,0,0.00,...,1.0,4.0,5.0,1.0,2.0,30.0,96727.878252,0.0,4.0,1
438942,24,10041911,21,1,6080.331558,1,0,0,0,1.00,...,1.0,4.0,5.0,1.0,2.0,30.0,96727.878252,0.0,4.0,1
438943,24,10041911,36,2,8019.906666,2,0,0,0,1.00,...,1.0,4.0,5.0,1.0,2.0,30.0,96727.878252,0.0,4.0,1


In [15]:
# df.dtypes

In [14]:
# df.info(memory_usage="deep")

In [27]:
df.loc[:, ['masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam',
       'gender_code', 'age', 'primary_card']].describe()

,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
count,438945.000000,438945.000000,438937.000000,438945.000000,438945.000000,438945.000000,4.160080e+05,435000.000000,435000.000000,438945.000000
mean,1.536630,3.629603,10.463590,1.001809,35.588037,29.478039,1.626480e+106,0.421510,3.661257,0.916015
std,0.500253,1.222454,4.776676,0.042493,45.659770,3.253548,3.162991e+108,0.493802,1.184319,0.277366
min,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.139239e+04,0.000000,1.000000,0.000000
25%,1.000000,3.000000,6.000000,1.000000,2.000000,30.000000,6.092366e+04,0.000000,3.000000,1.000000
50%,2.000000,3.000000,11.000000,1.000000,3.000000,30.000000,7.929124e+04,0.000000,4.000000,1.000000
75%,2.000000,5.000000,15.000000,1.000000,99.000000,30.000000,1.023826e+05,1.000000,4.000000,1.000000
max,3.000000,6.000000,28.000000,2.000000,99.000000,35.000000,6.151170e+110,1.000000,8.000000,1.000000


1. df: 1~24，兩年資料。
2. chid: 需要預測的人數為50萬人(目前只取前1萬人)。
3. shop_tag: 需預測的類別為16種，平均每人在兩年內消費的種類為7.5種。
4. 每人每月有消費的類別，平均刷卡數2.75，平均消費大約9000。
5. 刷卡數主要以國內為主，國內刷卡數線下較高，國外刷卡數線上較高。刷卡金額的相對亦是如此。
6. 婚姻狀況: 1, 2, 3，根據個數與平均年齡，猜測1是已婚、2是未婚、3是未知
7. 年齡: 1~8級距
8. 性別: 0 ,1
9. 教育: 1~6級距，不確定定義
10. 行業別: 1~28
11. 國籍: 1本國、2外國
12. 職位別: 總數不確定，但99應該是未知的資料

In [23]:
df.head()

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1,10384918,10,1,1085.361456,0,0,0,1,0.00,...,2.0,3.0,15.0,1.0,99.0,30.0,38072.451849,0.0,3.0,1
1,1,10384918,48,4,8523.693513,0,0,0,4,0.00,...,2.0,3.0,15.0,1.0,99.0,30.0,38072.451849,0.0,3.0,1
2,1,10378106,2,2,2987.631549,2,0,0,0,1.00,...,2.0,5.0,5.0,1.0,99.0,30.0,48209.277501,1.0,3.0,1
3,1,10378106,12,2,8690.365708,1,1,0,0,0.25,...,2.0,5.0,5.0,1.0,99.0,30.0,48209.277501,1.0,3.0,1
4,1,10011300,2,3,4388.591772,3,0,0,0,1.00,...,1.0,2.0,13.0,1.0,3.0,30.0,112010.611717,1.0,4.0,1


In [1]:
# df.groupby(['chid'])['shop_tag'].nunique().reset_index(name='shop_tag_n')\
#     ['shop_tag_n'].mean()
# df.groupby(['masts']).agg({'age': ['count', 'mean']})
# df.groupby(['educd']).agg({'age': ['count', 'mean']})
# df.groupby(['naty']).agg({'age': ['count', 'mean']})
# df.groupby(['poscd']).agg({'slam': ['count', 'mean']})


In [26]:
df.columns

Index(['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt',
       'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
       'domestic_offline_amt_pct', 'domestic_online_amt_pct',
       'overseas_offline_amt_pct', 'overseas_online_amt_pct', 'card_1_txn_cnt',
       'card_2_txn_cnt', 'card_3_txn_cnt', 'card_4_txn_cnt', 'card_5_txn_cnt',
       'card_6_txn_cnt', 'card_7_txn_cnt', 'card_8_txn_cnt', 'card_9_txn_cnt',
       'card_10_txn_cnt', 'card_11_txn_cnt', 'card_12_txn_cnt',
       'card_13_txn_cnt', 'card_14_txn_cnt', 'card_other_txn_cnt',
       'card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
       'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct',
       'card_7_txn_amt_pct', 'card_8_txn_amt_pct', 'card_9_txn_amt_pct',
       'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct',
       'card_13_txn_amt_pct', 'card_14_txn_amt_pct', 'card_other_txn_amt_pct',
       'masts', 'educd', 'trdtp', 'naty

In [2]:
# df.query('txn_cnt < 0').loc[:, ['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt',
#        'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
#        'domestic_offline_amt_pct', 'domestic_online_amt_pct',
#        'overseas_offline_amt_pct', 'overseas_online_amt_pct']]